<a href="https://colab.research.google.com/github/bradleywjenks/CIVE_70019_70057/blob/main/notebooks/hydraulic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Newton-Raphson method for hydraulic modelling
### CIVE 70019/70057
Department of Civil and Environmental Engineering, Imperial College London

### Preamble
This notebook walks through code implementation for developing a solver to simulate hydraulics in water distribution networks (WDNs). Specifically, we cover the following items:
- Using the `wntr` Python package for WDN analysis
- System of mass and energy conservation equations
- Newton-Raphson method for solving system of nonlinear equations
- Block factorisation and the Schur complement
- Application of hydraulic solver on different case networks
- Remarks on computational performance
- Simulated v. measured pressure head comparison using an operational network

First, we must clone the repository and install dependencies.

In [7]:
# Run this cell once
import sys
import os

if 'google.colab' in sys.modules:
    !git clone https://github.com/bradleywjenks/CIVE_70019_70057.git
    !pip install wntr
    !apt-get install libsuitesparse-dev && pip install scikit-sparse

fatal: destination path 'CIVE_70019_70057' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsuitesparse-dev is already the newest version (1:5.10.1+dfsg-4build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


### WNTR Python package

The Water Network Tool for Resilience (WNTR) Python package for simulating WDNs. It is based on the EPANET solvers. Here, we will be using `wntr` to load network data (.inp file) and compare hydraulic simulation results with our own solver.

#### Using WNTR's hydraulic solver
The code below follows an example from the `wntr` documentation: https://wntr.readthedocs.io/en/latest/index.html. Note that we have uploaded four (4) network (.inp) files for use in this notebook.

In [8]:
# Load dependencies
import wntr

# net_name = 'demo.inp'
net_name = 'stkl.inp'

# Create water network model
if 'google.colab' in sys.modules:
    # if run in Google Colab
    data_dir = '/content/CIVE_70019_70057/data/parameter_estimation/'
    net_dir = '/content/CIVE_70019_70057/data/networks/'
    print("Hello")
else:
    # replace with local directory
    data_dir = '/home/bradw/workspace/CIVE_70019_70057/data/parameter_estimation/'
    net_dir = '/home/bradw/workspace/CIVE_70019_70057/data/networks/'

wn = wntr.network.WaterNetworkModel(os.path.join(net_dir, net_name))

# Graph the network
wntr.graphics.plot_network(wn, title='Network layout', node_size=30)

# Simulate hydraulics
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

# Plot results on the network
hydraulic_state = 'head' # or 'flow' or 'head'

if hydraulic_state == 'head':
    head = results.node['head'].loc[0, :]
    wntr.graphics.plot_network(wn, node_attribute=head,
                            title='Hydraulic head', node_colorbar_label='Head [m]', node_size=30)
elif hydraulic_state == 'pressure':
    pressure = results.node['pressure'].loc[0, :]
    wntr.graphics.plot_network(wn, node_attribute=pressure,
                            title='Pressure head', node_colorbar_label='Pressure [m]', node_size=30)
elif hydraulic_state == 'flow':
    flow = results.link['flowrate'].loc[0, :]
    wntr.graphics.plot_network(wn, link_attribute=flow, title='Link flow', link_colorbar_label='Flow [L/s]', node_size=0)

ImportError: cannot import name '_center' from 'numpy._core.umath' (/usr/local/lib/python3.12/dist-packages/numpy/_core/umath.py)

#### Data extraction
We will now use WNTR to extract network data for developing our own hydraulic solver.

In [ ]:
# Load additional dependencies
import networkx as nx
import pandas as pd
import numpy as np
import scipy.sparse as sp

Get network elements and simulation information.

In [ ]:
'''
    nt = number of discrete time steps
    nn = number of demand or junction nodes
    np = number of links
    n0 = number of source (reservoir) nodes
'''

nt = int(wn.options.time.duration / wn.options.time.report_timestep)
nt = nt if nt>0 else 1
net_info = dict(
    np=wn.num_links,
    nn=wn.num_junctions,
    n0=wn.num_reservoirs,
    nt=nt,
    headloss=wn.options.hydraulic.headloss,
    units=wn.options.hydraulic.inpfile_units,
    reservoir_names=wn.reservoir_name_list,
    junction_names=wn.junction_name_list,
    pipe_names=wn.pipe_name_list,
    valve_names=wn.valve_name_list,
    prv_names=wn.prv_name_list
)

Extract link data.

In [ ]:
# Check head loss model
if net_info['headloss'] == 'H-W':
    n_exp = 1.852
elif net_info['headloss'] == 'D-W':
    n_exp = 2

# NB: 'C' denotes roughness or HW coefficient for pipes and local (minor) loss coefficient for valves
link_df = pd.DataFrame(
    index=pd.RangeIndex(net_info['np']),
    columns=['link_ID', 'link_type', 'diameter', 'length', 'n_exp', 'C', 'node_out', 'node_in'],
)
def link_dict(link):
    if isinstance(link, wntr.network.Pipe):  # check if the link is a pipe
        return dict(
            link_ID=link.name,
            link_type='pipe',
            diameter=link.diameter,
            length=link.length,
            n_exp=n_exp,
            C=link.roughness,
            node_out=link.start_node_name,
            node_in=link.end_node_name
        )
    elif isinstance(link, wntr.network.Valve): # check if the link is a valve
        return dict(
            link_ID=link.name,
            link_type='valve',
            diameter=link.diameter,
            length=2*link.diameter,
            n_exp=2,
            C=link.minor_loss,
            node_out=link.start_node_name,
            node_in=link.end_node_name
        )

for idx, link in enumerate(wn.links()):
    link_df.loc[idx] = link_dict(link[1])

link_df

,link_ID,link_type,diameter,length,n_exp,C,node_out,node_in
0,link_0001,pipe,0.1016,22.58,1.852,120.0,node_1916,node_0001
1,link_0002,pipe,0.1016,0.85,1.852,130.0,node_0001,node_0003
2,link_0003,pipe,0.1016,76.86,1.852,120.0,node_0001,node_1931
3,link_0004,pipe,0.1016,70.97,1.852,130.0,node_0002,node_1918
4,link_0005,pipe,0.1016,54.97,1.852,130.0,node_1918,node_1921
...,...,...,...,...,...,...,...,...
929,link_2536,valve,0.1016,0.2032,2,0.2,node_1631,node_1630
930,link_2537,valve,0.0766,0.1532,2,0.2,node_1634,node_1635
931,link_2538,valve,0.0508,0.1016,2,0.2,node_1643,node_1642
932,link_2545,valve,0.1,0.2,2,0.2,node_1663,node_1662


Extract node data.

In [ ]:
node_df = pd.DataFrame(
    index=pd.RangeIndex(wn.num_nodes), columns=["node_ID", "elev", "xcoord", "ycoord"]
)

def node_dict(node):
    if isinstance(node, wntr.network.elements.Reservoir):
        elev = 0
    else:
        elev = node.elevation
    return dict(
        node_ID=node.name,
        elev=elev,
        xcoord=node.coordinates[0],
        ycoord=node.coordinates[1]
    )

for idx, node in enumerate(wn.nodes()):
    node_df.loc[idx] = node_dict(node[1])

node_df

,node_ID,elev,xcoord,ycoord
0,node_0001,37.1254,874.76,34592.58
1,node_0002,37.12,877.47,34618.37
2,node_0003,37.12,876.31,34606.78
3,node_0010,36.57,1926.02,39214.03
4,node_0011,36.57,1932.22,39224.91
...,...,...,...,...
904,node_2455,59.91,66659.17,44697.87
905,node_2460,62.65,67760.08,30555.77
906,node_2462,60.01,68547.55,45148.35
907,node_2481,65.88,71902.66,30941.18


Create network graph.

In [ ]:
# extract network as a graph
uG = nx.from_pandas_edgelist(link_df, source='node_out', target='node_in')
pos = {row['node_ID']: (row['xcoord'], row['ycoord']) for _, row in node_df.iterrows()}
reservoir_labels = {node: 'Reservoir' for node in net_info['reservoir_names']}

# draw entire network
nx.draw(uG, pos, node_size=25, node_shape='o', node_color='black')

# draw reservoir nodes
nx.draw_networkx_nodes(uG, pos, nodelist=net_info['reservoir_names'], node_size=75, node_shape='s', node_color='black')

# reservoir labels
labels = nx.draw_networkx_labels(uG, pos, reservoir_labels, font_size=12, verticalalignment='bottom')
for _, label in labels.items():
    label.set_y(label.get_position()[1] + 80)

# create adjacency matrices
A = np.zeros((net_info['np'], net_info['nn']+net_info['n0']), dtype=int)
for k, row in link_df.iterrows():
    # find start node
    out_name = row['node_out']
    out_idx = node_df[node_df['node_ID']==out_name].index[0]
    # find end node
    in_name = row['node_in']
    in_idx = node_df[node_df['node_ID']==in_name].index[0]

    A[k, out_idx] = -1
    A[k, in_idx] = 1

junction_idx = node_df.index[node_df['node_ID'].isin(net_info['junction_names'])].tolist()
reservoir_idx = node_df.index[node_df['node_ID'].isin(net_info['reservoir_names'])].tolist()

# incident matrices
A12 = A[:, junction_idx]; A12 = sp.csr_matrix(A12) # link-junction incident matrix
A10 = A[:, reservoir_idx]; A10 = sp.csr_matrix(A10) # link-reservoir indicent matrix

Extract operational data (e.g. demands, boundary heads).

In [ ]:
# demand data
demand_df = results.node['demand'].T
col_names = [f'demands_{t}' for t in range(1, len(demand_df.columns)+1)]
demand_df.columns = col_names
demand_df.reset_index(drop=False, inplace=True)
demand_df = demand_df.rename(columns={'name': 'node_ID'})

if net_info['nt'] > 1:
    demand_df = demand_df.iloc[:, :-1] # delete last time step

demand_df = demand_df[~demand_df['node_ID'].isin(net_info['reservoir_names'])] # delete reservoir nodes
demand_df

In [ ]:
# boundary head data
h0_df = results.node['head'].T
col_names = [f'h0_{t}' for t in range(1, len(h0_df.columns)+1)]
h0_df.columns = col_names
h0_df.reset_index(drop=False, inplace=True)
h0_df = h0_df.rename(columns={'name': 'node_ID'})

if net_info['nt'] > 1:
    h0_df = h0_df.iloc[:, :-1] # delete last time step

h0_df = h0_df[h0_df['node_ID'].isin(net_info['reservoir_names'])] # only reservoir nodes
h0_df

<span style="color:red">**NB: The above code comprise functions 'load_network_data' and 'plot_network' in the file '~/src/general_functions.py'**.</span>

### Newton-Raphson hydraulic solver
We will now develop our own hydraulic solver using the Newton-Raphson method for solving nonlinear systems of equations.

#### System of nonlinear equations
Hydraulic states are computed for each time step $t \in \{1, \dots, n_t\}$, where $n_t$ denotes the number of discrete time steps in the simulation period. These comprise the following unknown variables for which we wish to solve:
- Flow rate $q_t \in \mathbb{R}^{n_p}$ conveyed across network links, where $n_p$ is the number of links
- Hydraulic head $h_t \in \mathbb{R}^{n_n}$ at network nodes, where $n_n$ is the number of junction nodes

Their solution is governed by **energy** and **mass** conservation equations, as follows:

\begin{equation}
    A_{11}(q_t)q_t + A_{12}h_t + A_{10}h_{0t} = 0 \\
    A_{12}^{T}q_t + d_t = 0,
\label{eq:hydraulic} \tag{1}
\end{equation}

where
- $A_{11} \in \mathbb{R}^{n_p \times n_p} = R_j|q_j^{n-1}|, \;\, \forall j \in \{1, \dots, n_p\}$ is a diagonal matrix of frictional head losses and $R_j$ is the pipe or valve resistance coefficient
- $A_{12} \in \mathbb{R}^{n_n \times n_p}$ is the node-link incidence matrix
- $A_{21} (=A_{12}^T) \in \mathbb{R}^{n_p \times n_n}$ is the link-node incidence matrix
- $A_{10} \in \mathbb{R}^{n_p \times n_0}$ is the link-known head incidence matrix
- $d_t \in \mathbb{R}^{n_n}$ is the vector of known demands (i.e. loading conditions)
- $h_{0t} \in \mathbb{R}^{n_0}$ is the vector of known hydraulic heads (i.e. boundary conditions)

We omit time index $t$ henceforth since the same equations are applied for all time steps. The system of nonlinear equations in \eqref{eq:hydraulic} is written in matrix form, as follows:

\begin{equation}
 F(q,h):= \left[
    {\renewcommand{\arraystretch}{1.5}
        \begin{array}{cc}
            A_{11}(q) & A_{12}  \\
            A_{12}^{T} & 0 \\
        \end{array}}\right]
    \left[
    {\renewcommand{\arraystretch}{1.5}
        \begin{array}{c}
            q \\
            h \\
        \end{array}}\right] -
    \left[
    {\renewcommand{\arraystretch}{1.5}
        \begin{array}{c}
            -A_{10}h_0 \\
            d \\
        \end{array}}\right] = 0.
\label{eq:hydraulic_matrix} \tag{2}
\end{equation}

#### Newton-Raphson method
We can then apply the Newton-Raphson method to solve the system of nonlinear equations in \eqref{eq:hydraulic_matrix}. The Newton-Raphson method is an iterative numerical technique for finding the root of an equation. At each iteration $k$, a new estimate of the root is found using the following formula:

\begin{equation}
    x^{k+1} = x^k - J(x^k)^{-1}F(x^k),
\label{eq:newton_raphson_1} \tag{3}
\end{equation}

where $J(x)$ is the Jacobian, a matrix of first-order partial derivatives. In general form, if $F(x)$ has $m$ equations, $f_1,f_2,\dots,f_m$, and $n$ variables, $x_1,x_2,\dots,x_n$, as

\begin{equation}
    F(x)= \left[\begin{array}{c}
    f_1(x_1,\cdot,x_n)\\
    \vdots\\
    f_m(x_1,\cdot,x_n)\\
    \end{array}\right],
\label{eq:general_function} \tag{4}
\end{equation}

then the Jacobian $J(x)$ is expressed mathematically as follows:

\begin{equation}
    J(x) =\left[\begin{array}{cccc}
    \frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & \cdots & \frac{\partial f_1}{\partial x_n}\\
    \frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} & \cdots & \frac{\partial f_2}{\partial x_n}\\
    \vdots & \vdots & \ddots & \vdots\\
    \frac{\partial f_m}{\partial x_1} & \frac{\partial f_m}{\partial x_2} & \cdots & \frac{\partial f_m}{\partial x_n}\\
    \end{array}\right].
\label{eq:jacobian} \tag{5}
\end{equation}

#### Hydraulic solver
We now define the Jacobian $J(x)$ and Newton-Raphson steps to compute a unique solution to the hydraulic model in \eqref{eq:hydraulic}. The system of nonlinear equations,

\begin{equation}
F(q, h) := \left\lbrace \begin{array}{ll}
    f_1(q,h) = A_{11}(q)q + A_{12}h + A_{10}h_0 = 0 \\
    f_2(q,h) = A_{21}q - d = 0,
\end{array}\right.
\label{eq:nonlinear_system} \tag{6}
\end{equation}

has a Jacobian computed as

\begin{equation}
J (q, h) := \left\lbrace \begin{array}{ll}
    \frac{\partial f_1 }{\partial q}=\frac{\partial A_{11}(q)q}{\partial q}=\frac{\partial R|q^{n-1}|q}{\partial q}={N Kq}^{n-1} ={NA_{11}(q)} \\
    \frac{\partial f_1 }{\partial h}=\frac{\partial A_{12} h}{\partial h}=A_{12} \\
    \frac{\partial f_2 }{\partial q}=\frac{\partial A_{12}^T q}{\partial q}=A_{12}^T \\
    \frac{\partial f_2 }{\partial h}=0,
\end{array}\right.
\label{eq:jacobian_hydraulic} \tag{7}
\end{equation}

which can be written in matrix form as

\begin{equation}
    J(q,h)=\left[\begin{array}{cc}
    \frac{\partial f_1}{\partial q} & \frac{\partial f_1}{\partial h} \\
    \frac{\partial f_2}{\partial q} & \frac{\partial f_2}{\partial h} \\
    \end{array}\right]=\left[\begin{array}{cc}
    NA_{11}(q) & {A_{12}} \\
    {A_{12}^T} & 0 \\
    \end{array}\right], \; \text{where} \;
    N=\left[\begin{array}{cccc}
        n & 0 & \cdots & 0 \\
        0 & n & \cdots & 0 \\
        \vdots & \vdots & \ddots & 0 \\
        0 & 0 & \cdots & n \\
    \end{array}\right].
\label{eq:jacobian_hydraulic_matrix} \tag{8}
\end{equation}

However, computing the inverse of the Jacobian is not computationally efficient (at every iteration) and it may not be possible for singular matrices (i.e. zero-flow pipes). We therefore multiply both sides of \eqref{eq:newton_raphson_1} by $J(x^k)$ to eliminate its inverse:

\begin{equation}
    J(x^k)x^{k+1}=J(x^k)x^k-F(x^k).
\label{eq:newton_raphson_2} \tag{9}
\end{equation}

Each new iterate $x_{k+1}$ of the Newton-Raphson equation is computed by solving the following linear system:

\begin{equation}
    \left[\begin{array}{cc}
         NA_{11}(h_k)& A_{12} \\
         A_{12}^T& 0 \\
    \end{array}\right]
    \left[\begin{array}{c}
         q^{k+1} \\
         h^{k+1} \\
    \end{array}\right] =
    \left[\begin{array}{cc}
         NA_{11}(q^{k})& A_{12}  \\
         A_{12}^T& 0 \\
    \end{array}\right]
    \left[\begin{array}{c}
         q^{k} \\
         h^{k} \\
    \end{array}\right] - \left[\left[\begin{array}{cc}
         A_{11}(q^{k})& A_{12} \\
         A_{12}^T& 0\\
    \end{array}\right]
    \left[\begin{array}{c}
         q^{k} \\
         h^{k} \\
    \end{array}\right]+\left[\begin{array}{c}
         A_{10}h_0 \\
         -d\\
    \end{array}\right]\right],
\label{eq:newton_step_1} \tag{10}
\end{equation}

which simplifies to,

\begin{equation}
    \left[\begin{array}{cc}
         NA_{11}(q^k)& A_{12} \\
         A_{12}^T & 0\\
    \end{array}\right] \left[\begin{array}{c}
          q^{k+1} \\
          h^{k+1} \\
    \end{array}\right]=
    \left[\begin{array}{c}
         (N-I)A_{11}(q^k)q^k-A_{10}h_0\\
         d\\
    \end{array}\right].
\label{eq:newton_step_2} \tag{11}
\end{equation}

At each iteration $k$, we perform the following convergence check:

\begin{equation}
\begin{Vmatrix}
        A_{11}(q^{k+1})q^{k+1} + A_{12}h^{k+1} + A_{10}h_0 \\
        A_{12}^Tq^{k+1} - d
\end{Vmatrix}
    _{\infty} < \epsilon_{\text{tol}}
\label{eq:convergence_check} \tag{12}
\end{equation}

where the solver has converged if the maximum error is less than a specified tolerance $\epsilon_{\text{tol}}$. The solution process for using the Newton-Raphson method for solving the system of nonlinear equations is shown in the figure below.

In [ ]:
# code for generating solution process chart
from graphviz import Digraph

dot = Digraph()

# Add nodes
dot.node("start", label="Start", shape="rect", width="1.5", height="0.5", style="rounded")
dot.node("input", label="Input network data", shape="parallelogram")
dot.node("initial", label="Initialize q^k, h^k", shape="parallelogram")
dot.node("end", label="End", shape="rect", width="1.5", height="0.5", style="rounded")
dot.node("convergence", label="Convergence check", shape="diamond", width="1.0", height="0.75")
dot.node("compute_h", label="Compute h^{k+1}", shape="rect", width="2.5")
dot.node("compute_q", label="Compute q^{k+1}", shape="rect", width="2.5")
dot.node("update", label="Update A_11(q^{k+1})", shape="rect")
dot.node("convergence", label="Convergence check", shape="diamond", width="1.0", height="0.75")
dot.node("dummy_1", label="No", shape="plaintext", width="0.75", height="0.5")
dot.node("dummy_2", label="k = k + 1", shape="plaintext", width="0.75", height="0.5")
dot.node("dummy_3", label="Yes", shape="plaintext", width="0.75", height="0.5")

# Add edges
dot.edge("start", "input")
dot.edge("input", "initial")
dot.edge("convergence", "dummy_3", dir="none")
dot.edge("dummy_3", "end")
dot.edge("initial", "compute_h")
dot.edge("compute_h", "compute_q", headport="n", tailport="s")
dot.edge("compute_q", "convergence", headport="n", tailport="s")
dot.edge("dummy_1", "update", headport="s")
dot.edge("dummy_2", "update", dir="none", headport="n")

# Subgraphs
with dot.subgraph(name="same_level_1") as s1:
    s1.attr(rank="same")
    s1.edge("compute_q", "update", style="invis", minlen="3")
with dot.subgraph(name="same_level_2") as s2:
    s2.attr(rank="same")
    s2.edge("convergence", "dummy_1", dir="none")
with dot.subgraph(name="same_level_3") as s3:
    s3.attr(rank="same")
    s3.edge("compute_h", "dummy_2", dir="back", minlen="2")
with dot.subgraph(name="same_level_4") as s4:
    s4.attr(rankdir="TB")
    s4.edge("dummy_1", "dummy_2", style="invis")
with dot.subgraph(name="same_level_5") as s5:
    s5.attr(rank="TB")
    s5.edge("compute_h", "compute_q", style="invis")
    s5.edge("compute_q", "convergence", style="invis")

# View flowchart
dot

Below is code for implementing the Newton-Raphson solver described above.

First, we use previously written code (now in a function) to load network data. We have to load these functions as follows.

In [ ]:
# load functions from src folder
if 'google.colab' in sys.modules:
    sys.path.append('/content/CIVE_70019_70057/src/')
    from general_functions import *
    from hydraulic_functions import *
else:
    sys.path.append('/home/bradw/workspace/CIVE_70019_70057/src/')
    from general_functions import *
    from hydraulic_functions import *

Then, we can load network data using the previously defined function 'load_network_data'

In [ ]:
# net_name = 'demo.inp'
net_name = 'stkl.inp'

wdn = load_network_data(os.path.join(net_dir, net_name))

The network layout is plotted below.

In [ ]:
plot_network(wdn)

Code for the Newton-Raphson solver is below.

In [ ]:
### Step 1: unload network and hydraulic data
A12 = wdn.A12
A10 = wdn.A10
net_info = wdn.net_info
link_df = wdn.link_df
node_df = wdn.node_df
demand_df = wdn.demand_df
h0_df = wdn.h0_df

# define head loss equations
def friction_loss(net_info, df):
    if net_info['headloss'] == 'H-W':
        K = 10.67 * df['length'] * (df['C'] ** -df['n_exp']) * (df['diameter'] ** -4.8704)
    else:
        K = [] # insert DW formula here...

    return K

def local_loss(df):
    K = (8 / (np.pi ** 2 * 9.81)) * (df['diameter'] ** -4) * df['C']

    return K

# compute loss coefficients
K = np.zeros((net_info['np'], 1))
for idx, row in link_df.iterrows():
    if row['link_type'] == 'pipe':
        K[idx] = friction_loss(net_info, row)

    elif row['link_type'] == 'valve':
        K[idx] = local_loss(row)

n_exp = link_df['n_exp'].astype(float).to_numpy().reshape(-1, 1)

# set stopping criteria
tol = 1e-9
kmax = 50

# small values in A11 make convergence unsteady; therefore, we need to define a lower bound -- see Todini (1988), page 7
tol_A11 = 1e-9

# set solution arrays
q = np.zeros((net_info['np'], net_info['nt']))
h = np.zeros((net_info['nn'], net_info['nt']))


# run over all time steps
for t in range(net_info['nt']):

    ### Step 2: set initial values
    hk = 130 * np.ones((net_info['nn'], 1))
    qk = 0.03 * np.ones((net_info['np'], 1))

    # set boundary head and demand conditions
    dk = demand_df.iloc[:, t+1].to_numpy(); dk = dk.reshape(-1, 1)
    h0k = h0_df.iloc[:, t+1].to_numpy(); h0k = h0k.reshape(-1, 1)

    # initialise A11 matrix
    A11_diag = K * (abs(qk) ** (n_exp - 1)) # diagonal elements of matrix A11
    A11_diag[A11_diag < tol_A11] = tol_A11 # replace with small value = tol_A11
    A11 = sp.diags(A11_diag.T, [0]) # matrix A11, allocated as a sparse diagonal matrix

    # begin iterations
    for k in range(kmax):

        ### Step 3: compute h^{k+1} and q^{k+1} for each iteration k
        N = sp.diags(n_exp.T, [0]) # matrix N
        I = sp.eye(net_info['np'], format='csr') # identiy matrix with dimension np x np, allocated as a sparse matrix

        b = np.concatenate([(N - I) @ A11 @ qk - A10 @ h0k, dk])
        J = sp.bmat([[N @ A11, A12], [A12.T, sp.csr_matrix((net_info['nn'], net_info['nn']))]], format='csr')

        # solve linear system
        x = sp.linalg.spsolve(J, b)
        qk = x[:net_info['np']]; qk = qk.reshape(-1, 1)
        hk = x[net_info['np']:net_info['np'] + net_info['nn']];hk = hk.reshape(-1, 1)

        # update A11 matrix
        A11_diag = K * (abs(qk) ** (n_exp - 1)) # diagonal elements of matrix A11
        A11_diag[A11_diag < tol_A11] = tol_A11 # replace with small value = tol_A11
        A11 = sp.diags(A11_diag.T, [0]) # matrix A11, allocated as a sparse diagonal matrix


        ### Step 4: convergence check
        err = A11 @ qk + A12 @ hk + A10 @ h0k
        max_err = np.linalg.norm(err, np.inf)

        # print progress
        print(f"Time step t={t+1}, Iteration k={k}. Maximum energy conservation error is {max_err} m.")

        if max_err < tol:
            # if successful,  break from loop
            break

    q[:, t] = qk.T
    h[:, t] = hk.T

# convert results to pandas dataframe
column_names_q = [f'q_{t+1}' for t in range(net_info['nt'])]
q_df = pd.DataFrame(q, columns=column_names_q)
q_df.insert(0, 'link_ID', link_df['link_ID'])

column_names_h = [f'h_{t+1}' for t in range(net_info['nt'])]
h_df = pd.DataFrame(h, columns=column_names_h)
h_df.insert(0, 'link_ID', link_df['link_ID'])


<span style="color:red">**NB: The above code comprises a function 'hydraulic_solver' in the file '~/src/hydraulic_functions.py'**.</span>

### Newton-Raphson solver with Schur complement

We note that the system of linear equations in \eqref{eq:newton_step_2} has a saddle point form. We can leverage this special structure to improve the computational performance of the Newton-Raphson solver.

#### Saddle point problem
Let us first present the Newton-Raphson step as a general linear system with saddle point form, as folllows:

\begin{equation}
\left[\begin{array}{cc}
     A&B_{1}  \\
     B_{2}^T & C \\
\end{array}\right] \left[\begin{array}{c}
     x \\
    y \\
\end{array}\right]=\left[\begin{array}{c}
     f  \\
     g \\
\end{array}\right], \; \text{or} \; \mathcal{A}u=b
\label{eq:saddle_point} \tag{13}
\end{equation}

where the block matrices in \eqref{eq:saddle_point} must satisfy the following conditions:
1. $A \succeq 0$ is symmetric positive semidefinite
2. $B_1 = B_2 = B$
3. $C \succeq 0$ is symmetric positive semidefinite

In the Newton-Raphson step for our hydraulic simulation \eqref{eq:newton_step_2}, these variables are defined as:
- $A=NA_{11}(q_k)$
- $B = A_{12}$
- $C = O$ (zero matrix)
- $x = q^{k+1}$
- $y = h^{k+1}$
- $f = (N-I)A_{11}(q^k)q^k-A_{10}h_0$
- $g = d$

#### Block factorisation and the Schur complement
If A is nonsingular, the saddle point matrix $\mathcal{A}$ admits the following block triangular factorization:

\begin{equation}
    \mathcal{A}=\left[\begin{array}{cc}
     A&B_1  \\
     B_2^T& O \\
\end{array}\right] =\left[\begin{array}{cc}
     I & O \\
    B_2^TA^{-1} & I \\
\end{array}\right] \left[\begin{array}{cc}
     A & O  \\
     O & S \\
\end{array}\right] \left[\begin{array}{cc}
     I & A^{-1}B_1 \\
     O & I \\
\end{array}\right]
\label{saddle_property_1} \tag{14}
\end{equation}

\begin{equation}
    \mathcal{A} = \left[\begin{array}{cc}
     A & O  \\
     B_2^T & S \\
\end{array}\right] \left[\begin{array}{cc}
     I & A^{-1}B_1 \\
     O & I \\
\end{array}\right]
\label{saddle_property_2} \tag{15}
\end{equation}

\begin{equation}
    \mathcal{A} = \left[\begin{array}{cc}
     I & O  \\
     B_2^TA^{-1} & I \\
\end{array}\right] \left[\begin{array}{cc}
     A & B_1 \\
     O & S \\
\end{array}\right]
\label{saddle_property_3} \tag{16}
\end{equation}

where $S=-(B_2^TA^{-1}B_1)$ is the Schur complement of $A$ in $\mathcal{A}$. In our hydraulic simulation problem, the Schur complement is defined as $S=-A_{12}^T\left(NA_{11}(q^k)\right)^{-1}A_{12}$.

The following conditions must hold on the system of linear equations to ensure that $S$ and $\mathcal{A}$ are nonsingular:
1. $A$ is symmetric positive definite
2. $B$ has full column rank (i.e. $rank(B) = m$), where $m$ is the number of columns in $B$
2. $ker(A)\cap ker(B)={0}$

For the sake of brevity, we do not show a proof that $\mathcal{A}$ is nonsingular for our linear system described in \eqref{eq:newton_step_2}. We also note that, as introduced previously, it is common practice to include a small regularisation term $A_{11}{\text{_tol}}$ to the diagonal elements of $A_{11}$ to ensure it is positively defined (and is therefore invertible). This has negligible impact on the energy conservation of the network.

#### Application of Schur complement to Newton-Raphson solver
Recall the system of linear equations solved at each Newton-Raphson step,

\begin{equation}
    \left[\begin{array}{cc}
         NA_{11}(q^k)& A_{12} \\
         A_{12}^T & 0\\
    \end{array}\right] \left[\begin{array}{c}
          q^{k+1} \\
          h^{k+1} \\
    \end{array}\right]=
    \left[\begin{array}{c}
         (N-I)A_{11}(q^k)q^k-A_{10}h_0\\
         d\\
    \end{array}\right].
\label{eq:recall_newton_step} \tag{17}
\end{equation}

The above system can be written in a more expanded form

\begin{eqnarray}
   NA_{11}(q^k)q^{k+1}+A_{12}h^{k+1}&=&(N-I)A_{11}(q^k)q^k-A_{10}h_0 \label{eq:newton_step_expand_1} \tag{18}\\
    A_{12}^Tq^{k+1}&=&d \label{eq:newton_step_expand_2} \tag{19}
\end{eqnarray}

Then, multiplying $A_{12}^T\left(NA_{11}(q^k)\right)^{-1}$ to each side of \eqref{eq:newton_step_expand_1} and substituting $A_{12}^Tq^{k+1}=d$ yields

\begin{equation}
A_{12}^T\left(NA_{11}(q^k)\right)^{-1}A_{12}h^{k+1}=A_{12}^T\left(NA_{11}(q^k)\right)^{-1}\left((N-I)A_{11}(q^k)q^k-A_{10}h_0\right)-d
\tag{20}
\end{equation}

Finally, with $\left(NA_{11}(q^k)\right)^{-1}=N^{-1}\left(A_{11}(q^k)\right)^{-1}$, we have

\begin{equation}
A_{12}^TN^{-1}\left(A_{11}(q^k)\right)^{-1}A_{12}h^{k+1}=-A_{12}^TN^{-1}(q^k+\left(A_{11}(q^k)\right)^{-1}A_{10}h_0)+A_{12}^Tq^k-d
\label{eq:h_update} \tag{21}
\end{equation}

A new estimate of $h^{k+1}$ can then be computed. Here, the symmetric positive definite matrix $A_{12}^TN^{-1}\left(A_{11}(q^k)\right)^{-1}A_{12}$ (**Schur complement** of $NA_{11}^k$ in $\begin{bmatrix}
    N A_{11}^k & A_{12} \\
    A_{12}^T & 0 \\
\end{bmatrix} $) is much smaller $(n_n \times n_n)$ than the Jacobian $(n_p+n_n \times n_p+n_n)$ in the original Newton-Raphson step. It also has nice properties, allowing for the use of fast factorisation algorithms (e.g. Cholesky factorisation).

Once $h^{k+1}$ is determined, we can directly compute $q^{k+1}$ by rearanging terms and substituting in $h^{k+1}$, as follows

\begin{equation}
   q^{k+1}=-N^{-1}\left(A_{11}(q^k)\right)^{-1}A_{12}h^{k+1}+(I-N^{-1})q^k-N^{-1}\left(A_{11}(q^k)\right)^{-1}A_{10}h_0
\end{equation}

Note that this solver routine is known as the global gradient algorithm (GGA), or 'nodal' version of the original Newton-Raphson method. The GGA algorithm is used in EPANET's solver routine.

Code for implementing the Newton-Raphson solver with Schur complement is below.

In [ ]:
### Step 1: unload network and hydraulic data
A12 = wdn.A12
A10 = wdn.A10
net_info = wdn.net_info
link_df = wdn.link_df
node_df = wdn.node_df
demand_df = wdn.demand_df
h0_df = wdn.h0_df

# define head loss equations
def friction_loss(net_info, df):
    if net_info['headloss'] == 'H-W':
        K = 10.67 * df['length'] * (df['C'] ** -df['n_exp']) * (df['diameter'] ** -4.8704)
    else:
        K = [] # insert DW formula here...

    return K

def local_loss(df):
    K = (8 / (np.pi ** 2 * 9.81)) * (df['diameter'] ** -4) * df['C']

    return K

# compute loss coefficients
K = np.zeros((net_info['np'], 1))
for idx, row in link_df.iterrows():
    if row['link_type'] == 'pipe':
        K[idx] = friction_loss(net_info, row)

    elif row['link_type'] == 'valve':
        K[idx] = local_loss(row)

n_exp = link_df['n_exp'].astype(float).to_numpy().reshape(-1, 1)

# set stopping criteria
tol = 1e-5
kmax = 50

# small values in A11 make convergence unsteady; therefore, we need to define a lower bound -- see Todini (1988), page 7
tol_A11 = 1e-5

# set solution arrays
q = np.zeros((net_info['np'], net_info['nt']))
h = np.zeros((net_info['nn'], net_info['nt']))


# run over all time steps
for t in range(net_info['nt']):

    ### Step 2: set initial values
    hk = 130 * np.ones((net_info['nn'], 1))
    qk = 0.03 * np.ones((net_info['np'], 1))

    # set boundary head and demand conditions
    dk = demand_df.iloc[:, t+1].to_numpy(); dk = dk.reshape(-1, 1)
    h0k = h0_df.iloc[:, t+1].to_numpy(); h0k = h0k.reshape(-1, 1)

    # initialise A11 matrix
    A11_diag = K * (abs(qk) ** (n_exp - 1)) # diagonal elements of matrix A11
    A11_diag[A11_diag < tol_A11] = tol_A11 # replace with small value = tol_A11
    A11 = sp.diags(A11_diag.T, [0]) # matrix A11, allocated as a sparse diagonal matrix

    # begin iterations
    for k in range(kmax):

        ### Step 3: compute h^{k+1} and q^{k+1}
        inv_A11_diag = 1 / A11_diag; # diagonal elements of the inverse of A11
        inv_A11 = sp.diags(inv_A11_diag.T, [0]) # inverse of A11, allocated as a sparse, diagonal matrix

        inv_N = sp.diags(1/n_exp.T, [0]) # inverse of matrix N

        DD = inv_N @ inv_A11 # matrix inv_N * inv_A11

        b = -A12.T @ inv_N @ (qk + inv_A11 @ (A10 @ h0k)) + A12.T @ qk - dk # right-hand side of linear system for finding h^{k+1]
        A = A12.T @ DD @ A12 # Schur complement

        # solve linear system for h^{k+1]
        hk = sp.linalg.spsolve(A, b); hk = hk.reshape(-1, 1)

        # solve q^{k+1} by substitution
        I = sp.eye(net_info['np'], format='csr') # identiy matrix with dimension np x np, allocated as a sparse matrix
        qk = (I - inv_N) @ qk - DD @ ((A12 @ hk) + (A10 @ h0k))

        # update A11 matrix
        A11_diag = K * (abs(qk) ** (n_exp - 1)) # diagonal elements of matrix A11
        A11_diag[A11_diag < tol_A11] = tol_A11 # replace with small value = tol_A11
        A11 = sp.diags(A11_diag.T, [0]) # matrix A11, allocated as a sparse diagonal matrix

        ### Step 4: convergence check
        err = A11 @ qk + A12 @ hk + A10 @ h0k
        max_err = np.linalg.norm(err, np.inf)

        # print progress
        print(f"Time step t={t+1}, Iteration k={k}. Maximum energy conservation error is {max_err} m.")

        if max_err < tol:
            # if successful,  break from loop
            break

    q[:, t] = qk.T
    h[:, t] = hk.T

# convert results to pandas dataframe
column_names_q = [f'q_{t+1}' for t in range(net_info['nt'])]
q_df = pd.DataFrame(q, columns=column_names_q)
q_df.insert(0, 'link_ID', link_df['link_ID'])

column_names_h = [f'h_{t+1}' for t in range(net_info['nt'])]
h_df = pd.DataFrame(h, columns=column_names_h)
h_df.insert(0, 'link_ID', link_df['link_ID'])

<span style="color:red">**NB: The above code comprises a function 'hydraulic_solver' in the file '~/src/hydraulic_functions.py'**.</span>

### Hydraulic solver comparison

Here, we will compare the accuracy and computational times of our Newton-Raphson (original) and Newton-Raphson (with Schur complement) hydraulic solvers. We also check their accuracy in comparison to the EPANET solver engine used in the `wntr` package.

Again, we first import the functions we have written and load the network data.

In [ ]:
import time # for comparing cpu times

# net_name = 'demo.inp'
net_name = 'stkl.inp'

wdn = load_network_data(os.path.join(net_dir, net_name))

Simulate hydraulics and compare computational times.

In [ ]:
### hydraulic_solver(wdn, method=None, print_status=False)

In [ ]:
### Newton-Raphson solver
start_time = time.perf_counter()
q_nr, h_nr = hydraulic_solver(wdn, method='nr')
end_time = time.perf_counter()
cpu_time_nr = end_time - start_time
cpu_time_nr

In [ ]:
### Newton-Raphson with Schur complement solver
start_time = time.perf_counter()
q_schur, h_schur = hydraulic_solver(wdn, method='nr_schur')
end_time = time.perf_counter()
cpu_time_schur = end_time - start_time
cpu_time_schur

In [ ]:
### EPANET solver (via WNTR package)
start_time = time.perf_counter()
q_epa, h_epa = epanet_solver(os.path.join(net_dir, net_name))
end_time = time.perf_counter()
cpu_time_epa = end_time - start_time
cpu_time_epa

The results indicate an increase in computational speed applying the Schur complement in the Newton-Raphson step.

Note that the EPANET solver used in `wntr` is faster since it is compiled in C++. Python is known to have a slower compiling time since it is a higher level programming language. Moreover, the presented code was not intended to have efficiency as its main goal. This could certainly be improved upon. In any case, the purpose of this notebook was to describe the solver routine for hydraulic models.

We also show how to plot node and edge values below using the `networkx` package.

In [ ]:
# set plotting values
t = 8
plot_type = 'pressure head' # 'flow' or 'hydraulic head' or 'pressure head'
if plot_type == 'hydraulic head' or plot_type == 'pressure head':
    vals = h_schur
elif plot_type == 'flow':
    vals = q_schur

plot_network(wdn, plot_type=plot_type, vals=vals, t=t)

An accuracy comparison with EPANET is also presented below.

In [ ]:
import matplotlib.pyplot as plt

# Newton-Raphson v. EPANET: flow
qdiff_1 = q_nr.iloc[:, 1:] - q_epa.iloc[:, 1:]
qdiff_1_array = qdiff_1.values.flatten() * 1000
plt.hist(qdiff_1_array, bins = 25, color='blue', alpha=0.6)

# Set labels and title
plt.xlabel('Flow difference [L/s]', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Newton-Raphson v. EPANET', fontsize=14)

# Display the plot
plt.show()

In [ ]:
# Newton-Raphson v. EPANET: hydraulic head
hdiff_1 = h_nr.iloc[:, 1:] - h_epa.iloc[:, 1:]
hdiff_1_array = hdiff_1.values.flatten()
plt.hist(hdiff_1_array, bins = 25, color='red', alpha=0.6)

# Set labels and title
plt.xlabel('Head difference [m]', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Newton-Raphson v. EPANET', fontsize=14)

# Display the plot
plt.show()

In [ ]:
# Newton-Raphson with Schur v. EPANET: flow
qdiff_2 = q_schur.iloc[:, 1:] - q_epa.iloc[:, 1:]
qdiff_2_array = qdiff_2.values.flatten() * 1000
plt.hist(qdiff_2_array, bins = 25, color='blue', alpha=0.6)

# Set labels and title
plt.xlabel('Flow difference [L/s]', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Newton-Raphson with Schur v. EPANET', fontsize=14)

# Display the plot
plt.show()

In [ ]:
# Newton-Raphson with Schur v. EPANET: hydraulic head
hdiff_2 = h_schur.iloc[:, 1:] - h_epa.iloc[:, 1:]
hdiff_2_array = hdiff_2.values.flatten()
plt.hist(hdiff_2_array, bins = 25, color='red', alpha=0.6)

# Set labels and title
plt.xlabel('Head difference [m]', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Newton-Raphson with Schur v. EPANET', fontsize=14)

# Display the plot
plt.show()

### Future work items

This notebook could be extended to include:
1. Tanks (i.e. storage facilities with finite volume)
2. Hydraulic controls (e.g. pressure control valves, pumping)

### Field data comparison

Load field data.

In [ ]:
net_name = 'stkl.inp'
data_name = 'stkl_coursework_dataset.npy'

data = np.load(os.path.join(data_dir, data_name), allow_pickle=True).item()
h_data = data['h_data']
sensor_idx = data['sensor_idx'] - 1
d_data = data['d']
h0_data = data['h0'].reshape(-1, 1).T

Redefine hydraulic solver function to input demand and h0 field data.

In [ ]:
# redefine hydraulic solver function to input demand and h0 data
def hydraulic_solver(inp_file, d_data, h0_data, C=None, demand=False):

    # load network from wntr
    inp_file = os.path.join(net_dir, net_name)
    wn = wntr.network.WaterNetworkModel(inp_file)

    # get network properties
    reservoir_names = wn.reservoir_name_list
    junction_names = wn.junction_name_list
    link_names = wn.link_name_list

    # modify simulation time and hydraulic time step
    nt = h0_data.shape[1]
    wn.options.time.duration = (nt - 1) * 3600
    wn.options.time.hydraulic_timestep = 3600
    wn.options.time.pattern_timestep = 3600
    wn.options.time.report_timestep = 3600

    # assign reservoir data
    for (i, name) in enumerate(reservoir_names):
        wn.add_pattern(f'h0_{i}', h0_data[i])
        reservoir = wn.get_node(name)
        reservoir.head_timeseries.base_value = 1
        reservoir.head_timeseries.pattern_name = wn.get_pattern(f'h0_{i}')

    # replace demand data
    for idx, name in enumerate(junction_names):

        if any(val != 0 for val in d_data[idx, :]):
            node = wn.get_node(name)
            d_pat = d_data[idx, :]
            wn.add_pattern('d_'+name, d_pat)

            for (i, num) in enumerate(node.demand_timeseries_list):
                if i == 0:
                    node.demand_timeseries_list[i].base_value = 1
                    node.demand_timeseries_list[i].pattern_name = 'd_'+name
                else:
                    node.demand_timeseries_list[i].base_value = None
                    node.demand_timeseries_list[i].pattern_name = None

    # assign roughness (or HW) coefficients
    if C is not None:
        for name, link in wn.links():

            # check if the link is a pipe
            if isinstance(link, wntr.network.Pipe):
                link.roughness = C[link_names.index(name)]

            # check if link is a valve
            elif isinstance(link, wntr.network.Valve):
                link.minor_loss = C[link_names.index(name)]
                link.initial_setting = C[link_names.index(name)]

    # run simulation and get results
    sim = wntr.sim.EpanetSimulator(wn)
    results = sim.run_sim()

    q_sim = results.link['flowrate'].T
    h_sim = results.node['head'].T
    h_sim = h_sim[~h_sim.index.isin(reservoir_names)] # delete reservoir nodes
    d = results.node['demand'].T
    d = d[~d.index.isin(reservoir_names)] # delete reservoir nodes


    if demand == True:
        return d.to_numpy()
    else:
        return q_sim.to_numpy(), h_sim.to_numpy()

Simulate hydraulics.

In [ ]:
_, h = hydraulic_solver(os.path.join(net_dir, net_name), d_data, h0_data)

Compute pressure residuals.

In [ ]:
residuals = h[sensor_idx, :] - h_data

fig, ax = plt.subplots()
boxplot = ax.boxplot(residuals.T, medianprops=dict(color="red", linewidth=1.0), flierprops=dict(marker="+", markeredgecolor="red"), whiskerprops=dict(linestyle=(5, (8, 5)), linewidth=0.6), boxprops=dict(color='black', linewidth=0.6), capprops=dict(linewidth=0.6))
ax.set_xlabel('Sensor index', fontsize=12)
ax.set_ylabel('Pressure residual [m]', fontsize=12)

### Questions
1. Familiarise yourself with the networks, data structures and hydraulic simulation functions (this will be useful for the coursework!). For NR and NR with Schur, see if you can recognize the steps discussed in class.
2. Try running the hydraulic simulation for STKLnet using (a) NR and (b) NR with Schur. Comment on the differences in computation times observed.
3. For STKLnet, compare simulated and measured pressures at All_Sensors. Specifically, comment on the sign (i.e. are pressures over or underestimated in the hydraulic simulation?) and temporal (i.e. are the residuals time dependent?) and spatial distribution (i.e. are all sensors affected?) of the pressure residuals? How do you interpret this difference?